In [130]:
import pandas as pd
import numpy as np

from sklearn.linear_model import LinearRegression
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split, cross_validate, KFold, LeaveOneOut
from sklearn.preprocessing import StandardScaler, normalize, MinMaxScaler, PowerTransformer

from scipy.stats.mstats import winsorize

import matplotlib.pyplot as plt
from matplotlib.ticker import FuncFormatter

from datetime import datetime

import seaborn as sns

# Data processing

## Get data

In [131]:
df_raw = pd.read_csv('exploratory2.csv')

print(df_raw.dtypes)

ID_PATH_FLOWMONTH           int64
ISO                        object
PEAKTYPE                   object
HEDGETYPE                  object
SOURCEID                    int64
SINKID                      int64
CLUSTERID_LOW_SRC          object
CLUSTERID_LOW_SNK          object
SOURCE_ZONE                object
SINK_ZONE                  object
MW_NET                    float64
MW_BUY                    float64
MW_SELL                   float64
DC_MCP_WAVG               float64
DC_AUCTION_TYPE_FIRST      object
DC_AUCTION_MONTH_FIRST     object
FW_DC_FIRST                 int64
FLOWMONTH                  object
AUCTION_ID                 object
AUCTION_MONTH              object
AUCTION_TYPE               object
FW                          int64
MCP                       float64
CONG                      float64
FUTURE_AUCTION_ID          object
FUTURE_AUCTION_MONTH       object
FUTURE_AUCTION_TYPE        object
FUTURE_FW                 float64
FUTURE_MCP                float64
MAXMCP_LAST18 

In [132]:
df_raw.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_FW,FUTURE_MCP,MAXMCP_LAST18,MINPOSMCP_LAST18,WINRATE_BUY_LASTMARK,WINRATE_SELL_LASTMARK,MEDIANCONG,MEDIANCONG_INSEASON,CONG_LOW_T4,CONG_TOP_T4
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,0.0,1.05,7.950000,0.830000,0.68,0.32,-3.888,-10.1472,-279.18560,8.432400
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.0,19.41,162.527273,12.700000,0.00,1.00,16.452,8.2636,-17.45090,147.895175
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,1.0,20.41,162.527273,12.700000,0.00,1.00,16.452,8.2636,-17.45090,147.895175
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.0,19.41,162.527273,12.700000,0.44,0.56,18.220,8.2636,-17.45090,147.895175
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,0.0,-4.56,5.980000,0.277021,0.96,0.04,-3.890,-7.2012,-52.92265,5.036275


In [133]:
df_raw.describe()

,ID_PATH_FLOWMONTH,SOURCEID,SINKID,MW_NET,MW_BUY,MW_SELL,DC_MCP_WAVG,FW_DC_FIRST,FW,MCP,...,FUTURE_FW,FUTURE_MCP,MAXMCP_LAST18,MINPOSMCP_LAST18,WINRATE_BUY_LASTMARK,WINRATE_SELL_LASTMARK,MEDIANCONG,MEDIANCONG_INSEASON,CONG_LOW_T4,CONG_TOP_T4
count,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.023091e+06,...,2.019640e+06,2.019640e+06,2.018881e+06,1.780339e+06,2.023091e+06,2.023091e+06,2.023091e+06,2.014954e+06,1.969123e+06,1.985510e+06
mean,3.260349e+16,4.384865e+08,4.472235e+08,1.054111e+01,1.428585e+01,3.744738e+00,2.616037e+01,8.869535e+00,5.589818e+00,3.299981e+01,...,2.311615e+00,2.940816e+01,2.694552e+02,4.605098e+01,4.197102e-01,5.802898e-01,2.043855e+01,2.446035e+01,-3.867724e+02,4.168736e+02
std,5.322809e+18,6.926265e+08,6.915405e+08,1.837164e+01,1.640692e+01,8.016405e+00,3.213635e+02,2.451281e+00,2.603479e+00,3.396788e+02,...,2.196493e+00,3.232869e+02,7.921377e+02,1.782004e+02,2.970548e-01,2.970548e-01,2.732802e+02,3.319475e+02,1.075307e+03,9.445375e+02
min,-9.223338e+18,4.882400e+04,4.882400e+04,-1.828000e+02,0.000000e+00,0.000000e+00,-5.760430e+03,2.000000e+00,1.000000e+00,-6.740720e+03,...,0.000000e+00,-1.097964e+04,-3.804930e+03,3.413330e-03,0.000000e+00,0.000000e+00,-6.821336e+03,-8.599746e+03,-5.381230e+04,1.360000e-02
25%,-4.577779e+18,2.160179e+07,3.106543e+07,0.000000e+00,4.200000e+00,0.000000e+00,-2.519202e+01,7.000000e+00,4.000000e+00,-2.441000e+01,...,0.000000e+00,-2.505000e+01,1.766000e+01,2.170000e+00,1.600000e-01,3.333330e-01,-1.422960e+01,-2.160200e+01,-3.937804e+02,5.261960e+01
50%,8.058980e+16,3.836801e+07,4.446086e+07,8.000000e+00,9.100000e+00,0.000000e+00,-3.110000e+00,9.000000e+00,6.000000e+00,1.720000e+00,...,2.000000e+00,2.110000e+00,7.451000e+01,7.780000e+00,4.400000e-01,5.600000e-01,9.520000e-01,9.600000e-01,-1.472692e+02,1.507448e+02
75%,4.634187e+18,1.084391e+09,1.084391e+09,1.650000e+01,1.880000e+01,5.100000e+00,1.611641e+01,1.100000e+01,8.000000e+00,3.515000e+01,...,4.000000e+00,3.731000e+01,2.371100e+02,2.690000e+01,6.666670e-01,8.400000e-01,2.066880e+01,2.826000e+01,-4.755593e+01,4.003368e+02
max,9.223273e+18,2.156114e+09,2.156114e+09,4.718000e+02,4.718000e+02,1.828000e+02,1.337885e+04,1.200000e+01,1.100000e+01,1.369067e+04,...,1.000000e+01,1.780315e+04,2.361044e+04,5.497150e+03,1.000000e+00,1.000000e+00,9.951554e+03,1.628471e+04,-8.000000e-03,2.374710e+04


## Placeholder for subsetting

In [134]:
df_sample = df_raw[
    (df_raw['MW_NET'] != 0) & ## Drop paths that they bought up and totally exited (like an annual flip)
    (df_raw['DC_MCP_WAVG'] != 0) &
    (df_raw['HEDGETYPE'] == 'Obligation')
].copy()

df_sample = df_sample.filter(items=[
    'ID_PATH_FLOWMONTH',
    'ISO',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'CLUSTERID_LOW_SRC',
    'CLUSTERID_LOW_SNK',
    'SOURCE_ZONE',
    'SINK_ZONE',
    'MW_NET',
    'MW_BUY',
    'MW_SELL',
    'DC_MCP_WAVG',
    'DC_AUCTION_TYPE_FIRST',
    'DC_AUCTION_MONTH_FIRST',
    'FW_DC_FIRST',
    'FLOWMONTH',
    # 'AUCTION_ID',
    'AUCTION_MONTH',
    'AUCTION_TYPE',
    'FW',
    'MCP',
    'CONG',
    # 'FUTURE_AUCTION_ID',
    'FUTURE_AUCTION_MONTH',
    'FUTURE_AUCTION_TYPE',
    'FUTURE_FW',
    'FUTURE_MCP',
    # 'MAXMCP_LAST18',
    # 'MINPOSMCP_LAST18',
    'WINRATE_BUY_LASTMARK',
    'WINRATE_SELL_LASTMARK',
    # 'MEDIANCONG',
    # 'MEDIANCONG_INSEASON',
    # 'CONG_LOW_T4',
    # 'CONG_TOP_T4'
])

df_sample.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,AUCTION_TYPE,FW,MCP,CONG,FUTURE_AUCTION_MONTH,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP,WINRATE_BUY_LASTMARK,WINRATE_SELL_LASTMARK
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,Mar,1,-0.07,0.2016,2023-03-01,Apr,0.0,1.05,0.68,0.32
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,Jun,2,23.79,46.4896,2023-07-01,Aug,0.0,19.41,0.00,1.00
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,Jun,2,23.79,46.4896,2023-06-01,Jul,1.0,20.41,0.00,1.00
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,Jul,1,20.41,46.4896,2023-07-01,Aug,0.0,19.41,0.44,0.56
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,Sep,3,-5.88,-0.4024,2023-11-01,Dec,0.0,-4.56,0.96,0.04


# Apply bid strategy

In [135]:
df_bids = df_sample.copy()
df_bids['AUCTION_MONTH'] = pd.to_datetime(df_bids['AUCTION_MONTH'], format='%Y-%m-%d')
df_bids['FUTURE_AUCTION_MONTH'] = pd.to_datetime(df_bids['FUTURE_AUCTION_MONTH'], format='%Y-%m-%d')
df_bids['FLOWMONTH'] = pd.to_datetime(df_bids['FLOWMONTH'], format='%Y-%m-%d')

## Want to bid at a "premium" to DC: means bidding higher on long and bidding lower on short
conditions = [
    (df_bids['MW_BUY'] >= df_bids['MW_SELL']) & (df_bids['DC_MCP_WAVG'] > 0), ## DC Buy, PF MCP --> go long
    (df_bids['MW_BUY'] >= df_bids['MW_SELL']) & (df_bids['DC_MCP_WAVG'] < 0), ## DC Buy, CF MCP --> go short
    (df_bids['MW_BUY'] < df_bids['MW_SELL']) & (df_bids['DC_MCP_WAVG'] > 0), ## DC Sell, PF MCP --> go short
    (df_bids['MW_BUY'] < df_bids['MW_SELL']) & (df_bids['DC_MCP_WAVG'] < 0) ## DC Sell, CF MCP --> go long
]

choices = [
    (df_bids['DC_MCP_WAVG'] * 1.2), ## Long: bid 20% higher than DC's clearing price
    (df_bids['DC_MCP_WAVG'] * 0.8), ## Short: bid 20% lower than DC's clearing price
    (df_bids['DC_MCP_WAVG'] * 0.8), ## Short: bid 20% lower than DC's clearing price
    (df_bids['DC_MCP_WAVG'] * 1.2), ## Long: bid 20% higher than DC's clearing price
]

df_bids['BID_ENTRY'] = np.select(conditions, choices, default=np.nan)
df_bids = df_bids[~df_bids['BID_ENTRY'].isna()]

df_bids['ENTRY_TYPE'] = np.where(df_bids['MW_BUY'] >= df_bids['MW_SELL'], 'BUY', 'SELL')
df_bids['FLOW_TYPE'] = np.where(df_bids['MCP'] >= 0, 'PF', 'CF')

## Get an indicator for whether entry cleared
df_bids['CLEARED_ENTRY'] = np.where(
    (df_bids['ENTRY_TYPE'] == 'BUY') & (df_bids['BID_ENTRY'] > df_bids['MCP']) | 
    (df_bids['ENTRY_TYPE'] == 'SELL') & (df_bids['BID_ENTRY'] < df_bids['MCP']),
    1,
    0
).astype(int)

df_bids.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_AUCTION_MONTH,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP,WINRATE_BUY_LASTMARK,WINRATE_SELL_LASTMARK,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,2023-03-01,Apr,0.0,1.05,0.68,0.32,-12.912000,BUY,CF,0
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-07-01,Aug,0.0,19.41,0.00,1.00,26.358201,BUY,PF,1
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-06-01,Jul,1.0,20.41,0.00,1.00,26.358201,BUY,PF,1
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,2023-07-01,Aug,0.0,19.41,0.44,0.56,26.358201,BUY,PF,1
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,2023-11-01,Dec,0.0,-4.56,0.96,0.04,-21.560000,BUY,CF,0


In [136]:
## We can also tabulate exits now. If our bid clear, the OFFER_EXIT is based on MCP (which is entry cost)

# df_potential_trades['OFFER_EXIT'] = df_potential_trades['MCP_ENTRY'] * 1.2
# df_potential_trades['CLEARED_EXIT'] = (df_potential_trades['OFFER_EXIT'] < df_potential_trades['FUTURE_MCP']).astype(int) ## this logic assumes only selling a previous BUY

## Want to try to exit at a greater premium if we are more forward from flow
## If forward >= 2 --> 30% premium, FW1 --> 20%, FW0 --> exit at cost
df_bids['OFFER_PREMIUM'] = np.select(
    [df_bids['FUTURE_FW'] >= 2,
     df_bids['FUTURE_FW'] == 1,
     df_bids['FUTURE_FW'] == 0],
     [30, 20, 0]
) 

## Want to bid at a "premium" to DC: means bidding higher on long and bidding lower on short
conditions_exit = [
    (df_bids['ENTRY_TYPE'] == 'BUY') & (df_bids['FLOW_TYPE'] == 'PF'), ## Long
    (df_bids['ENTRY_TYPE'] == 'BUY') & (df_bids['FLOW_TYPE'] == 'CF'), ## Short
    (df_bids['ENTRY_TYPE'] == 'SELL') & (df_bids['FLOW_TYPE'] == 'PF'), ## Short
    (df_bids['ENTRY_TYPE'] == 'SELL') & (df_bids['FLOW_TYPE'] == 'CF') ## Long
]

choices_exit = [
    (df_bids['MCP'] * (1 + df_bids['OFFER_PREMIUM']/100)), ## Long: sell for more than we bought
    (df_bids['MCP'] * (1 - df_bids['OFFER_PREMIUM']/100)), ## Short: buy back for less than we bought
    (df_bids['MCP'] * (1 - df_bids['OFFER_PREMIUM']/100)), ## Short: buy back for less than we bought (want to buy it back for less than we sold it)
    (df_bids['MCP'] * (1 + df_bids['OFFER_PREMIUM']/100)), ## Long: sell for more than we bought (want someone to pay us more to take it back)
]

## Get an indicator for whether entry cleared
df_bids['OFFER_EXIT'] = np.select(conditions_exit, choices_exit, default=np.nan)
df_bids['EXIT_TYPE'] = np.where(
    df_bids['ENTRY_TYPE'] == 'BUY', 'SELL', 'BUY'
)
## Get an indicator for whether entry cleared
df_bids['CLEARED_EXIT'] = np.where(
    (df_bids['EXIT_TYPE'] == 'BUY') & (df_bids['OFFER_EXIT'] > df_bids['FUTURE_MCP']) | 
    (df_bids['EXIT_TYPE'] == 'SELL') & (df_bids['OFFER_EXIT'] < df_bids['FUTURE_MCP']),
    1,
    0
).astype(int)

df_bids.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,WINRATE_BUY_LASTMARK,WINRATE_SELL_LASTMARK,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_PREMIUM,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,0.68,0.32,-12.912000,BUY,CF,0,0,-0.070,SELL,1
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.00,1.00,26.358201,BUY,PF,1,0,23.790,SELL,0
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.00,1.00,26.358201,BUY,PF,1,20,28.548,SELL,0
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0.44,0.56,26.358201,BUY,PF,1,0,20.410,SELL,0
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,0.96,0.04,-21.560000,BUY,CF,0,0,-5.880,SELL,1


In [137]:
df_bids['WINRATE'] = np.where(
    df_bids['ENTRY_TYPE'] == 'BUY',
    df_bids['WINRATE_BUY_LASTMARK'],
    df_bids['WINRATE_SELL_LASTMARK']
)

df_bids['RANK_WINRATE'] = df_bids.groupby(['AUCTION_MONTH','FLOWMONTH'])['WINRATE'].rank(method="first", ascending=False).astype(int)

In [138]:
## Want to know whether Long / Short a given cluster
## Long / Short is determined by whether entered as a BUY or a SELL

df_bids['LONGSHORT_SRC'] = np.where(
    df_bids['ENTRY_TYPE'] == 'BUY',
    -1, ## short the source if you buy
    +1 ## long the source if you sell
)

df_bids['LONGSHORT_SNK'] = np.where(
    df_bids['ENTRY_TYPE'] == 'BUY',
    +1, ## long the sink if you buy
    -1 ## short the sink if you sell
)

df_bids['LONGCLUSTER'] = np.where(
    df_bids['LONGSHORT_SNK'] == 1,
    df_bids['CLUSTERID_LOW_SNK'],
    df_bids['CLUSTERID_LOW_SRC']   
)

df_bids['SHORTCLUSTER'] = np.where(
    df_bids['LONGSHORT_SNK'] == -1,
    df_bids['CLUSTERID_LOW_SNK'],
    df_bids['CLUSTERID_LOW_SRC']   
)

df_bids.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,OFFER_PREMIUM,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT,WINRATE,RANK_WINRATE,LONGSHORT_SRC,LONGSHORT_SNK,LONGCLUSTER,SHORTCLUSTER
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,0,-0.070,SELL,1,0.68,515,-1,1,PPL:92,51252
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0,23.790,SELL,0,0.00,2313,-1,1,AEP:274,ATSI:86
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,20,28.548,SELL,0,0.00,2314,-1,1,AEP:274,ATSI:86
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0,20.410,SELL,0,0.44,2937,-1,1,AEP:274,ATSI:86
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,0,-5.880,SELL,1,0.96,365,-1,1,COMED:268,COMED:76


# Get portfolio outcomes

## Get trades and holdings

In [139]:
## df_bids is really a big set of potential trades

## To look at our portfolio we will do the following iteratively for each AUCTION_MONTH:
## 1) Consider potential trades:
##      a) Consider path-flowmonths that are not in our holdings
##      b) Subset to FW >= 3
## 2) See what clears, add it to *Holdings*
## 3) Get costs and revenue
## 4) Move on to the next AUCTION_MONTH & repeat

In [140]:
df_bids.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,OFFER_PREMIUM,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT,WINRATE,RANK_WINRATE,LONGSHORT_SRC,LONGSHORT_SNK,LONGCLUSTER,SHORTCLUSTER
0,-9223337983664651781,PJMISO,WEPEAK,Obligation,51252,659771,51252,PPL:92,PPL,PPL,...,0,-0.070,SELL,1,0.68,515,-1,1,PPL:92,51252
1,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0,23.790,SELL,0,0.00,2313,-1,1,AEP:274,ATSI:86
2,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,20,28.548,SELL,0,0.00,2314,-1,1,AEP:274,ATSI:86
3,-9223234284026271016,PJMISO,OFF7X8,Obligation,1369012492,1348264767,ATSI:86,AEP:274,ATSI,AEP,...,0,20.410,SELL,0,0.44,2937,-1,1,AEP:274,ATSI:86
4,-9223116829435080393,PJMISO,OFF7X8,Obligation,32417779,33092303,COMED:76,COMED:268,COMED,COMED,...,0,-5.880,SELL,1,0.96,365,-1,1,COMED:268,COMED:76


In [141]:
## A function to itertively keep trades based on the following diversification criteria:
##      Remove trades we already have 
##      Only allow 1 Long and 1 Short trade per cluster-flowmonth

def select_trades_longshort(df):

    ## Sort the trade set by our ranking
    df = df.sort_values(by=['RANK_WINRATE', 'AUCTION_MONTH', 'FLOWMONTH'])

    ## Initialize an empty df to store the selected trades
    selected_trades = pd.DataFrame(columns=df.columns)

    ## Iterate through unique 'FLOWMONTH' groups
    for (auction, flow) in df.groupby(['AUCTION_MONTH', 'FLOWMONTH']).groups.keys():
        ## Get all trades for the current group
        group_trades = df[(df['AUCTION_MONTH'] == auction) & (df['FLOWMONTH'] == flow)].copy()

        ## While there are still potential trades in the group
        while not group_trades.empty:
            ## Select the first trade (highest ranked) from the sorted group
            selected_trade = group_trades.head(1).reset_index()
            selected_trades = pd.concat([selected_trades, selected_trade], ignore_index=True)

            ## Don't need this if we just drop similar trades immediately
            # ## Add the selected trade's long cluster and short cluster to those dfs
            # longcluster = np.where(
            #     selected_trade['LONGSHORT_SNK'] == 1,
            #     selected_trade['CLUSTERID_LOW_SNK'],
            #     selected_trade['CLUSTERID_LOW_SRC']
            # )
            # df_temp_longclusters = pd.concat([df_temp_longclusters, longcluster], ignore_index=True)

            # shortcluster = np.where(
            #     selected_trade['LONGSHORT_SNK'] == -1,
            #     selected_trade['CLUSTERID_LOW_SNK'],
            #     selected_trade['CLUSTERID_LOW_SRC']
            # )
            # df_temp_shortclusters = pd.concat([df_temp_shortclusters, shortcluster], ignore_index=True)

            ## Remove all trades that share the same long/short cluster
            group_trades = group_trades[
                ## Drop trades based on the cluster criteria
                (group_trades['LONGCLUSTER'] != selected_trade['LONGCLUSTER'].values[0]) &
                (group_trades['SHORTCLUSTER'] != selected_trade['SHORTCLUSTER'].values[0]) &
                ## Drop trades that share a node we are already trading in that flow month
                (group_trades['SOURCEID'] != selected_trade['SOURCEID'].values[0]) &
                (group_trades['SINKID'] != selected_trade['SOURCEID'].values[0]) &
                (group_trades['SOURCEID'] != selected_trade['SINKID'].values[0]) &
                (group_trades['SINKID'] != selected_trade['SINKID'].values[0])
            ]

    return selected_trades

In [142]:
sorted_auction_months = sorted(df_bids['AUCTION_MONTH'].unique())
df_holdings = pd.DataFrame(columns=df_bids.columns)

for auction_month in sorted_auction_months:

    ## 1) Consider potential trades
    ## 1a) Get broad set of potential trades that we are not already holding
    df_potential_trades = df_bids[
        (df_bids['AUCTION_MONTH'] == auction_month) & 
        (~df_bids['ID_PATH_FLOWMONTH'].isin(df_holdings['ID_PATH_FLOWMONTH']))
    ]

    # df_potential_trades[~df_potential_trades['FUTURE_FW'].isna()] ## don't drop these, just have to hold (unless we remap these nodes)

    ## 1b) Only consider trades with enough opportunities to sell
    df_potential_trades = df_potential_trades[df_potential_trades['FW']>=3]
    if df_potential_trades.empty: continue ## Need this because, for example, standing in March, there are no FWs satisfying FW > 3 (planning year is June-June)

    ## 1c) This is where we select trades / diversify
    ## 1c.i) This function keeps 1 long (and 1 short) trade per cluster
    df_selected_trades = select_trades_longshort(df_potential_trades)

    
    ## 2) See what clears and add it to holdings
    ## Only keep what clears
    df_selected_trades = df_selected_trades[df_selected_trades['CLEARED_ENTRY'] == 1]
    # df_selected_trades = df_potential_trades[df_potential_trades['CLEARED_ENTRY'] == 1]
    
    ## 3) Get where we exit trades
    ## Sort by CLEARED_EXIT and then FUTURE_AUCTION_MONTH. Gives us the first thing to clear or the first record
    df_sorted = df_selected_trades.sort_values(by=['ID_PATH_FLOWMONTH', 'AUCTION_MONTH', 'CLEARED_EXIT', 'FUTURE_AUCTION_MONTH'], ascending=[True, True, False, True])
    df_first_rows = df_sorted.groupby(['ID_PATH_FLOWMONTH', 'AUCTION_MONTH']).first().reset_index()
    ## Handling cases where everything is empty <-- (this shouldn't happen)
    df_first_rows = df_first_rows.dropna(how='all')
    if df_first_rows.empty: 
        print(f"Month with empty df_first_rows: {auction_month}")
        continue

    ## Add the kept trades to holdings
    df_holdings = pd.concat([df_holdings, df_first_rows]).reset_index(drop=True)



/var/folders/4j/hkr47ph95yg0gwf7gpdpbc600000gn/T/ipykernel_42322/2307359161.py:22: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  selected_trades = pd.concat([selected_trades, selected_trade], ignore_index=True)
/var/folders/4j/hkr47ph95yg0gwf7gpdpbc600000gn/T/ipykernel_42322/3645433592.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df_holdings = pd.concat([df_holdings, df_first_rows]).reset_index(drop=True)
/var/folders/4j/hkr47ph95yg0gwf7gpdpbc600000gn/T/ipykernel_42322/2307359161.py:22: 

In [145]:
## Get cost and revenue
df_holdings.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT,WINRATE,RANK_WINRATE,LONGSHORT_SRC,LONGSHORT_SNK,LONGCLUSTER,SHORTCLUSTER,index
0,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,-810.498,BUY,0,1.0,1,1,-1,METED:39,METED:39,80.0
1,-9220810376192278441,PJMISO,ONPEAK,Obligation,40243783,32418595,AEP:331,AEP:81,AEP,AEP,...,17.472,SELL,0,0.0,10022,-1,1,AEP:81,AEP:331,358.0
2,-9220066552863087576,PJMISO,WEPEAK,Obligation,27677106,19623499,APS:26,19623499,APS,APS,...,-84.292,BUY,1,1.0,1,1,-1,APS:26,19623499,460.0
3,-9219936064532111128,PJMISO,OFF7X8,Obligation,50641,1048039,PPL:52,PPL:21,PPL,PPL,...,26.628,BUY,1,1.0,1,1,-1,PPL:52,PPL:21,524.0
4,-9212397093433755705,PJMISO,OFF7X8,Obligation,51231,50769,51231,PENELEC:47,PENELEC,PENELEC,...,-35.610,BUY,1,1.0,11,1,-1,51231,PENELEC:47,1359.0


In [146]:
df_holdings['FLOWMONTH'].unique()

<DatetimeArray>
['2023-12-01 00:00:00', '2024-03-01 00:00:00', '2024-02-01 00:00:00',
 '2023-10-01 00:00:00', '2024-04-01 00:00:00', '2024-05-01 00:00:00',
 '2023-09-01 00:00:00', '2023-11-01 00:00:00', '2024-01-01 00:00:00']
Length: 9, dtype: datetime64[ns]

## Calculate revenue

In [147]:
df_revenue = df_holdings.filter(items=[
    'ID_PATH_FLOWMONTH',
    'ISO',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'CLUSTERID_LOW_SRC',
    'CLUSTERID_LOW_SNK',
    'SOURCE_ZONE',
    'SINK_ZONE',
    'FLOWMONTH',
    'AUCTION_MONTH',
    'AUCTION_TYPE',
    'FW',
    'MCP',
    'CONG',
    'FUTURE_AUCTION_MONTH',
    'FUTURE_AUCTION_TYPE',
    'FUTURE_FW',
    'FUTURE_MCP',
    'BID_ENTRY',
    'ENTRY_TYPE',
    'FLOW_TYPE',
    'CLEARED_ENTRY',
    'OFFER_EXIT',
    'EXIT_TYPE',
    'CLEARED_EXIT'
]).copy()

df_revenue.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT
0,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,Aug,4.0,-760.07,-870.494062,SELL,CF,1,-810.498,BUY,0
1,-9220810376192278441,PJMISO,ONPEAK,Obligation,40243783,32418595,AEP:331,AEP:81,AEP,AEP,...,Aug,7.0,3.36,14.435875,BUY,PF,1,17.472,SELL,0
2,-9220066552863087576,PJMISO,WEPEAK,Obligation,27677106,19623499,APS:26,19623499,APS,APS,...,Aug,6.0,-87.02,-113.545756,SELL,CF,1,-84.292,BUY,1
3,-9219936064532111128,PJMISO,OFF7X8,Obligation,50641,1048039,PPL:52,PPL:21,PPL,PPL,...,Aug,2.0,1.81,-43.531898,SELL,PF,1,26.628,BUY,1
4,-9212397093433755705,PJMISO,OFF7X8,Obligation,51231,50769,51231,PENELEC:47,PENELEC,PENELEC,...,Apr,0.0,-218.94,-122.617058,SELL,CF,1,-35.610,BUY,1


In [148]:
## Need to assert this
df_revenue[df_revenue['CLEARED_ENTRY']==0].head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,FUTURE_AUCTION_TYPE,FUTURE_FW,FUTURE_MCP,BID_ENTRY,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT


In [149]:
df_revenue['COST'] = df_revenue['MCP']

df_revenue['REVENUE_MONTH'] = np.where(
    df_revenue['CLEARED_EXIT']==1,
    df_revenue['FUTURE_AUCTION_MONTH'],
    df_revenue['FLOWMONTH']
)

df_revenue['REVENUE'] = np.where(
    df_revenue['CLEARED_EXIT']==1,
    df_revenue['FUTURE_MCP'],
    df_revenue['CONG']
)

df_revenue['PROFIT'] = np.where(
    df_revenue['ENTRY_TYPE'] == 'BUY',
    df_revenue['REVENUE'] - df_revenue['COST'],
    df_revenue['COST'] - df_revenue['REVENUE']
)

In [150]:
df_revenue.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT,COST,REVENUE_MONTH,REVENUE,PROFIT
0,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,SELL,CF,1,-810.498,BUY,0,-623.46,2023-12-01,-344.0512,-279.4088
1,-9220810376192278441,PJMISO,ONPEAK,Obligation,40243783,32418595,AEP:331,AEP:81,AEP,AEP,...,BUY,PF,1,17.472,SELL,0,13.44,2024-03-01,14.0992,0.6592
2,-9220066552863087576,PJMISO,WEPEAK,Obligation,27677106,19623499,APS:26,19623499,APS,APS,...,SELL,CF,1,-84.292,BUY,1,-64.84,2023-07-01,-87.0200,22.1800
3,-9219936064532111128,PJMISO,OFF7X8,Obligation,50641,1048039,PPL:52,PPL:21,PPL,PPL,...,SELL,PF,1,26.628,BUY,1,38.04,2023-07-01,1.8100,36.2300
4,-9212397093433755705,PJMISO,OFF7X8,Obligation,51231,50769,51231,PENELEC:47,PENELEC,PENELEC,...,SELL,CF,1,-35.610,BUY,1,-35.61,2024-03-01,-218.9400,183.3300


# Profit and Loss

## PnL by flow month

In [151]:
df_pnl_flowmonth = df_revenue.groupby('FLOWMONTH').agg(
    PROFIT = ('PROFIT', 'sum'),
    TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count')
).reset_index()

df_pnl_flowmonth = df_pnl_flowmonth.sort_values(by=['FLOWMONTH'])
df_pnl_flowmonth['PROFIT_CUMSUM'] = df_pnl_flowmonth['PROFIT'].cumsum()

df_pnl_flowmonth['PROFIT_SCALED'] = df_pnl_flowmonth['PROFIT'] / df_pnl_flowmonth['TRADE_COUNT']
df_pnl_flowmonth['PROFITSCALED_CUMSUM'] = df_pnl_flowmonth['PROFIT_SCALED'].cumsum()

df_pnl_flowmonth.head(12)

,FLOWMONTH,PROFIT,TRADE_COUNT,PROFIT_CUMSUM,PROFIT_SCALED,PROFITSCALED_CUMSUM
0,2023-09-01,-3212.9832,155,-3212.9832,-20.728924,-20.728924
1,2023-10-01,-2694.7072,294,-5907.6904,-9.165671,-29.894595
2,2023-11-01,-10379.3118,449,-16287.0022,-23.116507,-53.011102
3,2023-12-01,-4265.5392,581,-20552.5414,-7.341720,-60.352822
4,2024-01-01,59932.2420,719,39379.7006,83.354996,23.002174
5,2024-02-01,50993.3588,849,90373.0594,60.062849,83.065023
6,2024-03-01,27492.4680,1012,117865.5274,27.166470,110.231493
7,2024-04-01,29072.1580,1189,146937.6854,24.450932,134.682425
8,2024-05-01,57910.7368,1367,204848.4222,42.363377,177.045803


In [152]:
df_pnl_flowmonth = df_revenue.groupby(['FLOWMONTH']).agg(
    PROFIT_TOTAL = ('PROFIT', 'sum'),
    TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count'),
    PROFIT_PF = ('PROFIT', lambda x: x[df_revenue['FLOW_TYPE'] == 'PF'].sum()),
    TRADE_COUNT_PF = ('ID_PATH_FLOWMONTH', lambda x: x[df_revenue['FLOW_TYPE'] == 'PF'].count()),
    PROFIT_CF = ('PROFIT', lambda x: x[df_revenue['FLOW_TYPE'] == 'CF'].sum()),
    TRADE_COUNT_CF = ('ID_PATH_FLOWMONTH', lambda x: x[df_revenue['FLOW_TYPE'] == 'CF'].count())
).reset_index()

df_pnl_flowmonth = df_pnl_flowmonth.sort_values(by=['FLOWMONTH'])
df_pnl_flowmonth['PROFIT_TOTAL_CUMSUM'] = df_pnl_flowmonth['PROFIT_TOTAL'].cumsum()
df_pnl_flowmonth['PROFIT_PF_CUMSUM'] = df_pnl_flowmonth['PROFIT_PF'].cumsum()
df_pnl_flowmonth['PROFIT_CF_CUMSUM'] = df_pnl_flowmonth['PROFIT_CF'].cumsum()

## Return per MW
df_pnl_flowmonth['PROFIT_PER_MW'] = df_pnl_flowmonth['PROFIT_TOTAL'] / df_pnl_flowmonth['TRADE_COUNT']
df_pnl_flowmonth['PF_PROFIT_PER_MW'] = df_pnl_flowmonth['PROFIT_PF'] / df_pnl_flowmonth['TRADE_COUNT_PF']
df_pnl_flowmonth['CF_PROFIT_PER_MW'] = df_pnl_flowmonth['PROFIT_CF'] / df_pnl_flowmonth['TRADE_COUNT_CF']


## Try to format?
df_pnl_flowmonth['PROFIT_TOTAL'] = df_pnl_flowmonth['PROFIT_TOTAL'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_TOTAL_CUMSUM'] = df_pnl_flowmonth['PROFIT_TOTAL_CUMSUM'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_PF'] = df_pnl_flowmonth['PROFIT_PF'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_PF_CUMSUM'] = df_pnl_flowmonth['PROFIT_PF_CUMSUM'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_CF'] = df_pnl_flowmonth['PROFIT_CF'].apply(lambda x: "${:,.2f}".format(x))
df_pnl_flowmonth['PROFIT_CF_CUMSUM'] = df_pnl_flowmonth['PROFIT_CF_CUMSUM'].apply(lambda x: "${:,.2f}".format(x))

df_pnl_flowmonth['PROFIT_PER_MW'] = df_pnl_flowmonth['PROFIT_PER_MW'].apply(lambda x: "$/MW {:,.2f}".format(x))
df_pnl_flowmonth['PF_PROFIT_PER_MW'] = df_pnl_flowmonth['PF_PROFIT_PER_MW'].apply(lambda x: "$/MW {:,.2f}".format(x))
df_pnl_flowmonth['CF_PROFIT_PER_MW'] = df_pnl_flowmonth['CF_PROFIT_PER_MW'].apply(lambda x: "$/MW {:,.2f}".format(x))

df_pnl_flowmonth = df_pnl_flowmonth[
    ['FLOWMONTH', 
     'PROFIT_TOTAL', 'PROFIT_TOTAL_CUMSUM',
     'PROFIT_PER_MW',
     'PROFIT_PF', 'PROFIT_PF_CUMSUM', 'PF_PROFIT_PER_MW',
     'PROFIT_CF', 'PROFIT_CF_CUMSUM', 'CF_PROFIT_PER_MW',
     'TRADE_COUNT', 
     'TRADE_COUNT_PF', 'TRADE_COUNT_CF']
]

df_pnl_flowmonth.head(12)

,FLOWMONTH,PROFIT_TOTAL,PROFIT_TOTAL_CUMSUM,PROFIT_PER_MW,PROFIT_PF,PROFIT_PF_CUMSUM,PF_PROFIT_PER_MW,PROFIT_CF,PROFIT_CF_CUMSUM,CF_PROFIT_PER_MW,TRADE_COUNT,TRADE_COUNT_PF,TRADE_COUNT_CF
0,2023-09-01,"$-3,212.98","$-3,212.98",$/MW -20.73,"$-1,237.41","$-1,237.41",$/MW -16.07,"$-1,975.58","$-1,975.58",$/MW -25.33,155,77,78
1,2023-10-01,"$-2,694.71","$-5,907.69",$/MW -9.17,"$1,925.70",$688.30,$/MW 15.16,"$-4,620.41","$-6,595.99",$/MW -27.67,294,127,167
2,2023-11-01,"$-10,379.31","$-16,287.00",$/MW -23.12,"$-6,864.61","$-6,176.31",$/MW -37.51,"$-3,514.70","$-10,110.69",$/MW -13.21,449,183,266
3,2023-12-01,"$-4,265.54","$-20,552.54",$/MW -7.34,"$-2,628.21","$-8,804.52",$/MW -13.08,"$-1,637.33","$-11,748.02",$/MW -4.31,581,201,380
4,2024-01-01,"$59,932.24","$39,379.70",$/MW 83.35,"$52,421.70","$43,617.17",$/MW 183.29,"$7,510.55","$-4,237.47",$/MW 17.35,719,286,433
5,2024-02-01,"$50,993.36","$90,373.06",$/MW 60.06,"$25,167.26","$68,784.43",$/MW 79.90,"$25,826.10","$21,588.63",$/MW 48.36,849,315,534
6,2024-03-01,"$27,492.47","$117,865.53",$/MW 27.17,"$9,062.90","$77,847.33",$/MW 24.76,"$18,429.57","$40,018.20",$/MW 28.53,1012,366,646
7,2024-04-01,"$29,072.16","$146,937.69",$/MW 24.45,"$12,721.29","$90,568.62",$/MW 28.27,"$16,350.86","$56,369.06",$/MW 22.13,1189,450,739
8,2024-05-01,"$57,910.74","$204,848.42",$/MW 42.36,"$44,644.49","$135,213.12",$/MW 89.11,"$13,266.24","$69,635.31",$/MW 15.32,1367,501,866


## PnL by zone

In [153]:
## sometimes zone is missing and that throws off the calculation
print(df_revenue['SOURCE_ZONE'].unique())
print(df_revenue['SINK_ZONE'].unique())

['METED' 'AEP' 'APS' 'PPL' 'PENELEC' 'EKPC' 'DPL' 'COMED' 'DOM' 'DEOK'
 'BGE' None 'DAY' 'ATSI' 'PECO' 'PEPCO' 'PSEG' 'OVEC' 'DUQ' 'JCPL' 'AECO'
 'RECO']
['METED' 'AEP' 'APS' 'PPL' 'PENELEC' 'DPL' 'COMED' 'DOM' 'DEOK' 'BGE'
 'AECO' None 'PEPCO' 'ATSI' 'PSEG' 'JCPL' 'PECO' 'DAY' 'RECO' 'EKPC' 'DUQ'
 'OVEC']


In [154]:
# df_pnl_zone = df_revenue.copy().groupby('SINK_ZONE').agg(
#     PROFIT = ('PROFIT', 'sum'),
#     TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count')
# ).reset_index()

df_pnl_zone = df_revenue.copy().groupby('SOURCE_ZONE').agg(
    PROFIT = ('PROFIT', 'sum'),
    TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count')
).reset_index()

print(f"Are there duplicate IDs?: {df_revenue.duplicated(subset=['ID_PATH_FLOWMONTH']).sum()}")
print(f"Are there missing profits?: {df_revenue['PROFIT'].isnull().sum()}")

df_pnl_zone['PROFIT_TOTAL'] = df_pnl_zone['PROFIT'].sum()

df_pnl_zone['PROFIT_SCALED'] = df_pnl_zone['PROFIT'] / df_pnl_zone['TRADE_COUNT']

df_pnl_zone.head(50)

Are there duplicate IDs?: 0
Are there missing profits?: 0


,SOURCE_ZONE,PROFIT,TRADE_COUNT,PROFIT_TOTAL,PROFIT_SCALED
0,AECO,1575.8001,131,196254.1159,12.029008
1,AEP,41531.1963,820,196254.1159,50.647800
2,APS,36165.5220,433,196254.1159,83.523145
3,ATSI,10448.5469,359,196254.1159,29.104587
4,BGE,5138.7182,123,196254.1159,41.778197
5,COMED,43939.5348,814,196254.1159,53.979772
6,DAY,7650.9907,97,196254.1159,78.876193
7,DEOK,85.7513,210,196254.1159,0.408340
8,DOM,11395.1597,772,196254.1159,14.760570
9,DPL,-20312.5985,270,196254.1159,-75.231846


## PnL in a cash flow sense

In [48]:
df_revenue.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,CLUSTERID_LOW_SRC,CLUSTERID_LOW_SNK,SOURCE_ZONE,SINK_ZONE,...,ENTRY_TYPE,FLOW_TYPE,CLEARED_ENTRY,OFFER_EXIT,EXIT_TYPE,CLEARED_EXIT,COST,REVENUE_MONTH,REVENUE,PROFIT
0,-9222893321979192545,PJMISO,OFF7X8,Obligation,71856761,34887787,DOM:86,DOM:15,DOM,DOM,...,BUY,CF,1,-27.930,SELL,1,-39.90,2023-09-01,-26.41,13.49
1,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,METED:39,METED:39,METED,METED,...,SELL,CF,1,-748.152,BUY,1,-623.46,2023-10-01,-960.00,336.54
2,-9221875954374263028,PJMISO,WEPEAK,Obligation,51241,1356162213,JCPL:9,PSEG:12,JCPL,PSEG,...,BUY,CF,1,-22.940,SELL,1,-22.94,2024-03-01,-15.11,7.83
3,-9221774299757105218,PJMISO,ONPEAK,Obligation,135389799,338269,APS:66,338269,APS,PEPCO,...,BUY,PF,1,584.532,SELL,1,449.64,2023-07-01,846.35,396.71
4,-9221237361624466890,PJMISO,ONPEAK,Obligation,50403,2155502045,PSEG:40,PSEG:51,PSEG,PSEG,...,BUY,PF,1,109.824,SELL,1,84.48,2023-10-01,113.13,28.65


In [49]:
df_entry = df_revenue.filter(items=[
    'ID_PATH_FLOWMONTH',
    'ISO',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'FLOWMONTH',
    'ENTRY_TYPE',
    'AUCTION_MONTH',
    'COST'
]).copy()
df_entry.rename(columns={'AUCTION_MONTH': 'MONTH_CF'}, inplace=True)
df_entry.rename(columns={'COST': 'CASHFLOW'}, inplace=True)
## If it was a long FW position, cost is negative (out-flow)
df_entry['CASHFLOW'] = np.where(
    df_entry['ENTRY_TYPE'] == 'SELL',
    df_entry['CASHFLOW'],
    df_entry['CASHFLOW']*(-1)
)

df_exit = df_revenue.filter(items=[
    'ID_PATH_FLOWMONTH',
    'ISO',
    'PEAKTYPE',
    'HEDGETYPE',
    'SOURCEID',
    'SINKID',
    'FLOWMONTH',
    'ENTRY_TYPE',
    'REVENUE_MONTH',
    'REVENUE'
]).copy()
df_exit.rename(columns={'REVENUE_MONTH': 'MONTH_CF'}, inplace=True)
df_exit.rename(columns={'REVENUE': 'CASHFLOW'}, inplace=True)

## If it was a long FW position, revenue is positive (in-flow)
df_exit['CASHFLOW'] = np.where(
    df_exit['ENTRY_TYPE'] == 'BUY',
    df_exit['CASHFLOW'],
    df_exit['CASHFLOW']*(-1)
)

df_pnl_cashflow = pd.concat([df_entry, df_exit]).reset_index(drop=True)
df_pnl_cashflow.head()

,ID_PATH_FLOWMONTH,ISO,PEAKTYPE,HEDGETYPE,SOURCEID,SINKID,FLOWMONTH,ENTRY_TYPE,MONTH_CF,CASHFLOW
0,-9222893321979192545,PJMISO,OFF7X8,Obligation,71856761,34887787,2024-05-01,BUY,2023-05-01,39.90
1,-9222514428509594973,PJMISO,ONPEAK,Obligation,50754,1552843818,2023-12-01,SELL,2023-05-01,-623.46
2,-9221875954374263028,PJMISO,WEPEAK,Obligation,51241,1356162213,2024-04-01,BUY,2023-05-01,22.94
3,-9221774299757105218,PJMISO,ONPEAK,Obligation,135389799,338269,2024-05-01,BUY,2023-05-01,-449.64
4,-9221237361624466890,PJMISO,ONPEAK,Obligation,50403,2155502045,2024-05-01,BUY,2023-05-01,-84.48


In [50]:
df_pnl_cashflow = df_pnl_cashflow.groupby('MONTH_CF').agg(
    PROFIT = ('CASHFLOW', 'sum'),
    TRADE_COUNT = ('ID_PATH_FLOWMONTH', 'count')
).reset_index()

df_pnl_cashflow = df_pnl_cashflow.sort_values(by=['MONTH_CF'])
df_pnl_cashflow['PROFIT_CUMSUM'] = df_pnl_cashflow['PROFIT'].cumsum()

df_pnl_cashflow['PROFIT_SCALED'] = df_pnl_cashflow['PROFIT'] / df_pnl_cashflow['TRADE_COUNT']
df_pnl_cashflow['PROFITSCALED_CUMSUM'] = df_pnl_cashflow['PROFIT_SCALED'].cumsum()

df_pnl_cashflow.head(20)

,MONTH_CF,PROFIT,TRADE_COUNT,PROFIT_CUMSUM,PROFIT_SCALED,PROFITSCALED_CUMSUM
0,2023-05-01,-482997.6100,21995,-482997.6100,-21.959428,-21.959428
1,2023-06-01,-145194.5600,13533,-628192.1700,-10.728926,-32.688354
2,2023-07-01,-4960.9800,11723,-633153.1500,-0.423183,-33.111537
3,2023-08-01,199458.1300,8160,-433695.0200,24.443398,-8.668139
4,2023-09-01,170748.8888,7315,-262946.1312,23.342295,14.674156
5,2023-10-01,295144.8228,6924,32198.6916,42.626346,57.300502
6,2023-11-01,-61018.8838,5850,-28820.1922,-10.430578,46.869924
7,2023-12-01,116006.0280,4982,87185.8358,23.285032,70.154956
8,2024-01-01,-6406.3852,3239,80779.4506,-1.977890,68.177066
9,2024-02-01,-63093.7380,2165,17685.7126,-29.142604,39.034462


In [25]:
print(f"Sum of profit column from flow-month PnL: {df_pnl_flowmonth['PROFIT'].sum()}")
print(f"Sum of profit column from cash-flow-month PnL: {df_pnl_cashflow['PROFIT'].sum()}")

Sum of profit column from flow-month PnL: 834447.5658
Sum of profit column from cash-flow-month PnL: 834447.5658000002
